In [1]:
import cStringIO
import os,sys
import string
import adabas
from adabas.api import *
from adabas.datamap import *

In [18]:
from ..Util.Mask import *

ValueError: Attempted relative import in non-package

In [19]:
pwd

u'C:\\Python\\MyTools\\Nat2PyBath\\Adabas\\demo'

In [2]:
DBID=12;FNR=11 # EmplTel file
STARTISN=0
RCOUNT=5
# define the mapping of data in record buffer to attributes
# of EmpTel class

# create datamap object for Employees-Telephone-List
emp=Datamap('EmplTel',
    String('personnel_id',  8),
    String('firstname',    20),
    String('m_initial',    20),
    String('lastname',     20),
    String('birth',         8),
    String('country',       3),
    String('areacode',      6),
    String('phone',        15),
    String('department',    6),
    String('jobtitle',     25))

In [3]:
lines=''                                         
extraline=''                                     
alter=False                                      
c1=Adabas(rbl=256,fbl=64,sbl=32,vbl=128,ibl=0)   
c1.cb.dbid=DBID
c1.cb.fnr=FNR
c1.cb.cid='1010'                                 
c1.fb.value='AA,AC,AD,AE,AH,8,U,AL,AN,AM,AO,AP.' 


In [4]:
c1.cb.isn=STARTISN                                                                 
# use emp Datamap on record buffer                                                 
emp.buffer=c1.rb                                                                   
emp.offset=0                                                                       
                                                                                  
count=0                                                                            
                                                                                   
try:                                                                               
    for count in range(RCOUNT):                                                    
        c1.readByIsn(getnext=1)                                                    
                                                                                    
        lines += "{} - {:30} - {} - {} - {} - {} - {}\n".format(
                emp.personnel_id,
                string.capwords(emp.lastname+', '+emp.firstname+' '+emp.m_initial),
                date.fromordinal(int(emp.birth[2:])-364).strftime('%d/%m/%Y'),
                    emp.country,
                    emp.phone,
                    emp.department,
                    string.capwords(emp.jobtitle))
except DataEnd:
    extraline+= 'Sequential Read by ISN returned '+str(count)+' record(s).'
    pass
except DatabaseError, (line, apa):
    extraline+='</table><br>Database Error:'+line
                

In [5]:
print lines


50005800 - Adam, Simone                   - 30/01/1952 - F - 44864858 - VENT59 - Chef De Service
50005600 - Moreno, Humberto               - 28/09/1955 - F - 42457727 - VENT07 - Chef De Service
50005500 - Blond, Alexandre               - 26/06/1955 - F - 42452720 - VENT56 - Chef De Service
50005300 - Maiziere, Elisabeth            - 23/01/1952 - F - 46758304 - VENT54 - Chef De Service
50004900 - Caoudal, Albert                - 17/06/1942 - F - 44346166 - VENT06 - Chef De Service



In [6]:
print emp.birth

00709467


In [7]:
mask_date = lambda data, mask='%d-%m-%Y': date.fromordinal(int(data[2:])-364).strftime(mask)

In [9]:
mask_date(emp.birth, mask='%d/%m/%Y')

'17/06/1942'